In [1]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility
from math import sqrt

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import scipy.io
mat = scipy.io.loadmat('/home/aniruddha/Documents/data_10feature.mat')

arr = mat['TR2_10feature']
arr = np.array(arr)
arr = arr.reshape(-1)
print(arr.shape)
X_train = []
for i in range(0,14):
    for j in range(0,arr[i].shape[0]):
        X_train.append(arr[i][j])
X_train = np.array(X_train)
print(X_train.shape)
y_train = []
for i in range(0,arr.shape[0]):
    for j in range(0,arr[i].shape[0]):
        y_train.append(i)
y_train = np.array(y_train)
print(y_train.shape)
print(y_train[1])

arr1 = mat['TS2_10feature']
arr1 = np.array(arr1)
arr1 = arr1.reshape(-1)
print(arr1.shape)
X_test = []
for i in range(0,14):
    for j in range(0,arr1[i].shape[0]):
        X_test.append(arr1[i][j])
X_test = np.array(X_test)
print(X_test.shape)
y_test = []
for i in range(0,arr1.shape[0]):
    for j in range(0,arr1[i].shape[0]):
        y_test.append(i)
y_test = np.array(y_test)
print(y_test.shape)
print(y_test[1])

arr2 = mat['TS1_10feature']
arr2 = np.array(arr2)
arr2 = arr2.reshape(-1)
print(arr2.shape)
X_test1 = []
for i in range(0,14):
    for j in range(0,arr2[i].shape[0]):
        X_test1.append(arr2[i][j])
X_test1 = np.array(X_test1)
print(X_test1.shape)
y_test1 = []
for i in range(0,arr2.shape[0]):
    for j in range(0,arr2[i].shape[0]):
        y_test1.append(i)
y_test1 = np.array(y_test1)
print(y_test1.shape)
print(y_test1[1])


(14,)
(2621, 10)
(2621,)
0
(14,)
(627, 10)
(627,)
0
(14,)
(1252, 10)
(1252,)
0


In [3]:
print(X_train.max())
print(X_test.max())
print(X_test1.max())

9499
7890
6804


In [4]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_test1 = X_test1.astype('float32')
X_train = X_train/10000
X_test = X_test/10000
X_test1 = X_test1/10000
print(X_train.max())
print(X_test.max())
print(X_test1.max())

0.9499
0.789
0.6804


In [5]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state = 0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)
X_test1, y_test1 = shuffle(X_test1, y_test1, random_state=0)

In [6]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 8 # 1st layer num features
num_hidden_2 = 6 # 2nd layer num features (the latent dim)
num_input = 10 
num_classes = 14

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_uniform([num_input, num_hidden_1], minval=-4*np.sqrt(6.0/(num_input + num_hidden_1)), maxval=4*np.sqrt(6.0/(num_input + num_hidden_1)))),
    'encoder_h2': tf.Variable(tf.random_uniform([num_hidden_1, num_hidden_2], minval=-4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)), maxval=4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)))),
    'decoder_h1': tf.Variable(tf.random_uniform([num_hidden_2, num_hidden_1], minval=-4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)), maxval=4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)))),
    'decoder_h2': tf.Variable(tf.random_uniform([num_hidden_1, num_input], minval=-4*np.sqrt(6.0/(num_input + num_hidden_1)), maxval=4*np.sqrt(6.0/(num_input + num_hidden_1)))),
    'classifier1_h': tf.Variable(tf.random_uniform([num_hidden_2, 10], minval=-4*np.sqrt(6.0/(10 + num_hidden_2)), maxval=4*np.sqrt(6.0/(10 + num_hidden_2)))),
    'classifier_h': tf.Variable(tf.random_uniform([10, num_classes], minval=-4*np.sqrt(6.0/(10 + num_classes)), maxval=4*np.sqrt(6.0/(10 + num_classes)))),
}
biases = {
    'encoder_b1': tf.Variable(tf.truncated_normal([num_hidden_1])/sqrt(num_hidden_1)),
    'encoder_b2': tf.Variable(tf.truncated_normal([num_hidden_2])/sqrt(num_hidden_2)),
    'decoder_b1': tf.Variable(tf.truncated_normal([num_hidden_1])/sqrt(num_hidden_1)),
    'decoder_b2': tf.Variable(tf.truncated_normal([num_input])/sqrt(num_hidden_2)),
    'classifier1_b': tf.Variable(tf.truncated_normal([10])/sqrt(10)),
    'classifier_b': tf.Variable(tf.truncated_normal([num_classes])/sqrt(num_classes)),
}

In [7]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
classify1 = tf.nn.sigmoid(tf.add(tf.matmul(encoder_op, weights['classifier1_h']), biases['classifier1_b']))
label_pred = tf.nn.softmax(tf.add(tf.matmul(classify1, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [8]:
from keras.utils import np_utils
y_test1 = np_utils.to_categorical(y_test1)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test1.shape)

(2621, 14)
(1252, 14)


In [9]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 32
num_batch = 38

# Training
for i in range(0,2000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 38

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))
    print("Epoch:", (i + 1), "accuracy =", "{:.8f}".format(sess.run(accuracy, feed_dict={X: X_train, Y: y_train})))

Epoch: 1 cost = 0.26939280
Epoch: 1 accuracy = 0.07592522
Epoch: 2 cost = 0.26782528
Epoch: 2 accuracy = 0.07592522
Epoch: 3 cost = 0.26622004
Epoch: 3 accuracy = 0.07592522
Epoch: 4 cost = 0.26457331
Epoch: 4 accuracy = 0.07592522
Epoch: 5 cost = 0.26288014
Epoch: 5 accuracy = 0.07592522
Epoch: 6 cost = 0.26113480
Epoch: 6 accuracy = 0.07592522
Epoch: 7 cost = 0.25933103
Epoch: 7 accuracy = 0.07592522
Epoch: 8 cost = 0.25746260
Epoch: 8 accuracy = 0.07592522
Epoch: 9 cost = 0.25552384
Epoch: 9 accuracy = 0.07783289
Epoch: 10 cost = 0.25351006
Epoch: 10 accuracy = 0.04845479
Epoch: 11 cost = 0.25141793
Epoch: 11 accuracy = 0.02937810
Epoch: 12 cost = 0.24924605
Epoch: 12 accuracy = 0.06829455
Epoch: 13 cost = 0.24699505
Epoch: 13 accuracy = 0.07745136
Epoch: 14 cost = 0.24466799
Epoch: 14 accuracy = 0.07745136
Epoch: 15 cost = 0.24227027
Epoch: 15 accuracy = 0.07745136
Epoch: 16 cost = 0.23980988
Epoch: 16 accuracy = 0.07745136
Epoch: 17 cost = 0.23729748
Epoch: 17 accuracy = 0.0774513

Epoch: 161 cost = 0.15767603
Epoch: 161 accuracy = 0.23998474
Epoch: 162 cost = 0.15720645
Epoch: 162 accuracy = 0.24074781
Epoch: 163 cost = 0.15671971
Epoch: 163 accuracy = 0.24570775
Epoch: 164 cost = 0.15623380
Epoch: 164 accuracy = 0.25906143
Epoch: 165 cost = 0.15594155
Epoch: 165 accuracy = 0.27661198
Epoch: 166 cost = 0.15637211
Epoch: 166 accuracy = 0.28462419
Epoch: 167 cost = 0.15561774
Epoch: 167 accuracy = 0.28042731
Epoch: 168 cost = 0.15458987
Epoch: 168 accuracy = 0.27890119
Epoch: 169 cost = 0.15401328
Epoch: 169 accuracy = 0.27813810
Epoch: 170 cost = 0.15353771
Epoch: 170 accuracy = 0.28386113
Epoch: 171 cost = 0.15309171
Epoch: 171 accuracy = 0.29034719
Epoch: 172 cost = 0.15268734
Epoch: 172 accuracy = 0.30560854
Epoch: 173 cost = 0.15238591
Epoch: 173 accuracy = 0.31858069
Epoch: 174 cost = 0.15214488
Epoch: 174 accuracy = 0.32773751
Epoch: 175 cost = 0.15159653
Epoch: 175 accuracy = 0.32659289
Epoch: 176 cost = 0.15090024
Epoch: 176 accuracy = 0.32277757
Epoch: 1

Epoch: 319 accuracy = 0.61999238
Epoch: 320 cost = 0.09530348
Epoch: 320 accuracy = 0.62266308
Epoch: 321 cost = 0.09459570
Epoch: 321 accuracy = 0.62876767
Epoch: 322 cost = 0.09422916
Epoch: 322 accuracy = 0.62113696
Epoch: 323 cost = 0.09449232
Epoch: 323 accuracy = 0.61655855
Epoch: 324 cost = 0.09420917
Epoch: 324 accuracy = 0.62800455
Epoch: 325 cost = 0.09366128
Epoch: 325 accuracy = 0.63105685
Epoch: 326 cost = 0.09374396
Epoch: 326 accuracy = 0.62533385
Epoch: 327 cost = 0.09386494
Epoch: 327 accuracy = 0.62418926
Epoch: 328 cost = 0.09321870
Epoch: 328 accuracy = 0.63220143
Epoch: 329 cost = 0.09272861
Epoch: 329 accuracy = 0.62724149
Epoch: 330 cost = 0.09293859
Epoch: 330 accuracy = 0.61922932
Epoch: 331 cost = 0.09284436
Epoch: 331 accuracy = 0.63029379
Epoch: 332 cost = 0.09225746
Epoch: 332 accuracy = 0.63563526
Epoch: 333 cost = 0.09227120
Epoch: 333 accuracy = 0.62685996
Epoch: 334 cost = 0.09248431
Epoch: 334 accuracy = 0.62991226
Epoch: 335 cost = 0.09190765
Epoch: 3

Epoch: 493 cost = 0.07342722
Epoch: 493 accuracy = 0.65776420
Epoch: 494 cost = 0.07347073
Epoch: 494 accuracy = 0.67378861
Epoch: 495 cost = 0.07309350
Epoch: 495 accuracy = 0.67913008
Epoch: 496 cost = 0.07362284
Epoch: 496 accuracy = 0.67188096
Epoch: 497 cost = 0.07372599
Epoch: 497 accuracy = 0.67531478
Epoch: 498 cost = 0.07285865
Epoch: 498 accuracy = 0.67645937
Epoch: 499 cost = 0.07255454
Epoch: 499 accuracy = 0.67111790
Epoch: 500 cost = 0.07307317
Epoch: 500 accuracy = 0.66043496
Epoch: 501 cost = 0.07276318
Epoch: 501 accuracy = 0.67836702
Epoch: 502 cost = 0.07270393
Epoch: 502 accuracy = 0.67798549
Epoch: 503 cost = 0.07332819
Epoch: 503 accuracy = 0.67302555
Epoch: 504 cost = 0.07298316
Epoch: 504 accuracy = 0.67874855
Epoch: 505 cost = 0.07212193
Epoch: 505 accuracy = 0.67989320
Epoch: 506 cost = 0.07222909
Epoch: 506 accuracy = 0.66501337
Epoch: 507 cost = 0.07257459
Epoch: 507 accuracy = 0.67226249
Epoch: 508 cost = 0.07212432
Epoch: 508 accuracy = 0.67951167
Epoch: 5

Epoch: 641 cost = 0.06451725
Epoch: 641 accuracy = 0.69324684
Epoch: 642 cost = 0.06489141
Epoch: 642 accuracy = 0.68752384
Epoch: 643 cost = 0.06370287
Epoch: 643 accuracy = 0.69973290
Epoch: 644 cost = 0.06286782
Epoch: 644 accuracy = 0.70011449
Epoch: 645 cost = 0.06351643
Epoch: 645 accuracy = 0.69553608
Epoch: 646 cost = 0.06372814
Epoch: 646 accuracy = 0.69973290
Epoch: 647 cost = 0.06364313
Epoch: 647 accuracy = 0.70278519
Epoch: 648 cost = 0.06461736
Epoch: 648 accuracy = 0.68294543
Epoch: 649 cost = 0.06417738
Epoch: 649 accuracy = 0.69553608
Epoch: 650 cost = 0.06276290
Epoch: 650 accuracy = 0.69896984
Epoch: 651 cost = 0.06265063
Epoch: 651 accuracy = 0.70011449
Epoch: 652 cost = 0.06341213
Epoch: 652 accuracy = 0.70125908
Epoch: 653 cost = 0.06318075
Epoch: 653 accuracy = 0.70469284
Epoch: 654 cost = 0.06378290
Epoch: 654 accuracy = 0.69477296
Epoch: 655 cost = 0.06426136
Epoch: 655 accuracy = 0.68676078
Epoch: 656 cost = 0.06305054
Epoch: 656 accuracy = 0.70011449
Epoch: 6

...

Epoch: 1330 accuracy = 0.81381154
Epoch: 1331 cost = 0.02718962
Epoch: 1331 accuracy = 0.81343001
Epoch: 1332 cost = 0.02527633
Epoch: 1332 accuracy = 0.80999619
Epoch: 1333 cost = 0.02458821
Epoch: 1333 accuracy = 0.80579931
Epoch: 1334 cost = 0.02439336
Epoch: 1334 accuracy = 0.77794737
Epoch: 1335 cost = 0.02565795
Epoch: 1335 accuracy = 0.74360931
Epoch: 1336 cost = 0.02794513
Epoch: 1336 accuracy = 0.77832890
Epoch: 1337 cost = 0.02839695
Epoch: 1337 accuracy = 0.80694389
Epoch: 1338 cost = 0.02549410
Epoch: 1338 accuracy = 0.80999619
Epoch: 1339 cost = 0.02763129
Epoch: 1339 accuracy = 0.81266689
Epoch: 1340 cost = 0.02473190
Epoch: 1340 accuracy = 0.78023654
Epoch: 1341 cost = 0.02654037
Epoch: 1341 accuracy = 0.75772607
Epoch: 1342 cost = 0.02578529
Epoch: 1342 accuracy = 0.78824878
Epoch: 1343 cost = 0.02469945
Epoch: 1343 accuracy = 0.80808854
Epoch: 1344 cost = 0.02599054
Epoch: 1344 accuracy = 0.81571919
Epoch: 1345 cost = 0.02516178
Epoch: 1345 accuracy = 0.81533766
Epoch:

Epoch: 1502 cost = 0.02284236
Epoch: 1502 accuracy = 0.77107972
Epoch: 1503 cost = 0.02129541
Epoch: 1503 accuracy = 0.80808854
Epoch: 1504 cost = 0.02381903
Epoch: 1504 accuracy = 0.81266689
Epoch: 1505 cost = 0.02206909
Epoch: 1505 accuracy = 0.81648225
Epoch: 1506 cost = 0.01929450
Epoch: 1506 accuracy = 0.81075925
Epoch: 1507 cost = 0.01946211
Epoch: 1507 accuracy = 0.79015642
Epoch: 1508 cost = 0.02033537
Epoch: 1508 accuracy = 0.77413201
Epoch: 1509 cost = 0.02094301
Epoch: 1509 accuracy = 0.78824878
Epoch: 1510 cost = 0.02170760
Epoch: 1510 accuracy = 0.80770701
Epoch: 1511 cost = 0.02010952
Epoch: 1511 accuracy = 0.81533766
Epoch: 1512 cost = 0.01852657
Epoch: 1512 accuracy = 0.81648225
Epoch: 1513 cost = 0.01822533
Epoch: 1513 accuracy = 0.80083936
Epoch: 1514 cost = 0.02022001
Epoch: 1514 accuracy = 0.75276613
Epoch: 1515 cost = 0.02233980
Epoch: 1515 accuracy = 0.77336895
Epoch: 1516 cost = 0.02053106
Epoch: 1516 accuracy = 0.80274707
Epoch: 1517 cost = 0.02311221
Epoch: 151

Epoch: 1668 accuracy = 0.76001525
Epoch: 1669 cost = 0.02058255
Epoch: 1669 accuracy = 0.76955360
Epoch: 1670 cost = 0.01879585
Epoch: 1670 accuracy = 0.79549789
Epoch: 1671 cost = 0.01766692
Epoch: 1671 accuracy = 0.81533766
Epoch: 1672 cost = 0.01580380
Epoch: 1672 accuracy = 0.81762683
Epoch: 1673 cost = 0.01486070
Epoch: 1673 accuracy = 0.81800842
Epoch: 1674 cost = 0.01441149
Epoch: 1674 accuracy = 0.81495613
Epoch: 1675 cost = 0.01443409
Epoch: 1675 accuracy = 0.79626095
Epoch: 1676 cost = 0.01880590
Epoch: 1676 accuracy = 0.75619990
Epoch: 1677 cost = 0.02014521
Epoch: 1677 accuracy = 0.78595954
Epoch: 1678 cost = 0.01827224
Epoch: 1678 accuracy = 0.81037772
Epoch: 1679 cost = 0.01666663
Epoch: 1679 accuracy = 0.81724530
Epoch: 1680 cost = 0.01539841
Epoch: 1680 accuracy = 0.81915301
Epoch: 1681 cost = 0.01457573
Epoch: 1681 accuracy = 0.81495613
Epoch: 1682 cost = 0.01437835
Epoch: 1682 accuracy = 0.80312860
Epoch: 1683 cost = 0.01755321
Epoch: 1683 accuracy = 0.76306754
Epoch:

Epoch: 1836 accuracy = 0.81037772
Epoch: 1837 cost = 0.01242308
Epoch: 1837 accuracy = 0.81266689
Epoch: 1838 cost = 0.01300417
Epoch: 1838 accuracy = 0.81953454
Epoch: 1839 cost = 0.01162199
Epoch: 1839 accuracy = 0.81877148
Epoch: 1840 cost = 0.01359209
Epoch: 1840 accuracy = 0.76497519
Epoch: 1841 cost = 0.01931010
Epoch: 1841 accuracy = 0.77413201
Epoch: 1842 cost = 0.01517185
Epoch: 1842 accuracy = 0.79969478
Epoch: 1843 cost = 0.01218633
Epoch: 1843 accuracy = 0.81266689
Epoch: 1844 cost = 0.01248844
Epoch: 1844 accuracy = 0.81838995
Epoch: 1845 cost = 0.01226840
Epoch: 1845 accuracy = 0.82029760
Epoch: 1846 cost = 0.01114358
Epoch: 1846 accuracy = 0.81571919
Epoch: 1847 cost = 0.01242869
Epoch: 1847 accuracy = 0.77603966
Epoch: 1848 cost = 0.01875809
Epoch: 1848 accuracy = 0.77107972
Epoch: 1849 cost = 0.01592880
Epoch: 1849 accuracy = 0.79435331
Epoch: 1850 cost = 0.01307426
Epoch: 1850 accuracy = 0.80999619
Epoch: 1851 cost = 0.01269735
Epoch: 1851 accuracy = 0.81610072
Epoch:

Epoch: 1992 accuracy = 0.80961466
Epoch: 1993 cost = 0.00944022
Epoch: 1993 accuracy = 0.81075925
Epoch: 1994 cost = 0.01056884
Epoch: 1994 accuracy = 0.79893172
Epoch: 1995 cost = 0.01353213
Epoch: 1995 accuracy = 0.76497519
Epoch: 1996 cost = 0.01621318
Epoch: 1996 accuracy = 0.78977489
Epoch: 1997 cost = 0.01156548
Epoch: 1997 accuracy = 0.80045784
Epoch: 1998 cost = 0.01089730
Epoch: 1998 accuracy = 0.80618083
Epoch: 1999 cost = 0.00944256
Epoch: 1999 accuracy = 0.80808854
Epoch: 2000 cost = 0.00923105
Epoch: 2000 accuracy = 0.80961466


In [11]:
# on 1000 epoch
print(sess.run([accuracy, label_pred], feed_dict={X: X_train[1:2], Y: y_train[1:2]}))
print(sess.run([accuracy, label_pred], feed_dict={X: X_train[101:102], Y: y_train[101:102]}))
print(sess.run([accuracy], feed_dict={X: X_test1, Y: y_test1}))

[1.0, array([[4.4058628e-28, 4.0989268e-18, 2.3999667e-06, 3.0249916e-05,
        2.4349506e-04, 1.8039243e-03, 1.1627743e-22, 3.1445390e-01,
        1.2147462e-06, 1.9745128e-03, 4.0183330e-08, 5.8013774e-02,
        6.2347639e-01, 4.2090729e-08]], dtype=float32)]
[0.0, array([[2.3790563e-09, 5.0825714e-05, 5.7545259e-07, 9.5963664e-04,
        4.7627003e-07, 3.9177078e-01, 1.7350453e-21, 1.4431323e-10,
        6.0609102e-01, 9.5672985e-06, 1.5821250e-18, 1.2018994e-06,
        1.1158775e-03, 5.6680759e-26]], dtype=float32)]
[0.6269968]


In [47]:
# on 1000 epoch
a = X_train[0].reshape(1,10)
a1 = y_train[0].reshape(1,14)
b = X_train[600].reshape(1,10)
b1 = y_train[600].reshape(1,14)
print(a.shape)
print(sess.run(accuracy, feed_dict={X: a, Y: a1}))
print(sess.run(accuracy, feed_dict={X: b, Y: b1}))

(1, 10)
1.0
0.0
